# Perceptron multi capa con PyTorch

El diseño permite una cantidad de capas ocultas dinamicas

In [7]:
import torch
import torch.nn as nn

torch.set_default_dtype(torch.float64)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        layers = []
        in_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(in_size, hidden_size))
            layers.append(nn.Tanh())
            in_size = hidden_size
        layers.append(nn.Linear(in_size, output_size))
        layers.append(nn.Tanh())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)


In [58]:
import torch.optim as optim

def train_model(model, x, y, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        outputs = model(x)
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y.data, 1)
            accuracy = (predicted == actual).sum().item() / actual.size(0)
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

Ejemplo de problema de clasificación resuelto

In [59]:
import pandas as pd

df = pd.read_csv('./practica_4_data/p13_ej4_classif.csv', delimiter=',')
x = torch.tensor(df.iloc[:, :2].values, dtype=torch.float64)
z = torch.tensor(pd.get_dummies(df.iloc[:, -1]).values, dtype=torch.float64)

input_size = 2
hidden_sizes = [4]
output_size = z.size(1)

model = MLP(input_size, hidden_sizes, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 500

train_model(model, x, z, criterion, optimizer, num_epochs)

print("\nFirst 5 elements of the dataset (features and labels):")
print(df.head())

predicted_data = model(x).detach().numpy()
print("\nFirst 5 predicted values:")
predicted_classes = torch.argmax(torch.tensor(predicted_data), dim=1) + 1
print(predicted_classes[:5])

Epoch [100/500], Loss: 0.3758, Accuracy: 0.9870
Epoch [200/500], Loss: 0.2853, Accuracy: 0.9940
Epoch [300/500], Loss: 0.2645, Accuracy: 0.9970
Epoch [400/500], Loss: 0.2564, Accuracy: 0.9970
Epoch [500/500], Loss: 0.2524, Accuracy: 0.9970

First 5 elements of the dataset (features and labels):
          A         B   C
0 -0.056240  0.336536   1
1  0.093222  0.138700   1
2  2.242435  3.008840   2
3  0.005527 -0.138740   1
4 -2.143559  2.439455   2

First 5 predicted values:
tensor([1, 1, 2, 1, 2])


In [72]:
import pandas as pd

df = pd.read_csv('./music_data/features_30_sec.csv', delimiter=',')

x = torch.tensor(df.iloc[:, 2:-1].values, dtype=torch.float64)
z = torch.tensor(pd.get_dummies(df['label']).values, dtype=torch.float64)

input_size = x.shape[1]
hidden_sizes = [50, 25]
output_size = z.size(1)

print(f"Input size: {input_size}")
print(f"Hidden sizes: {hidden_sizes}")
print(f"Output size: {output_size}")

model = MLP(input_size, hidden_sizes, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1000

train_model(model, x, z, criterion, optimizer, num_epochs)

print("\nFirst 5 elements of the dataset (features and labels):")
print(df.head()['label'])

predicted_data = model(x).detach().numpy()
print("\nFirst 5 predicted values:")

predicted_classes = torch.argmax(torch.tensor(predicted_data), dim=1)
predicted_labels = df['label'].unique()[predicted_classes]
print(predicted_labels[:5])

Input size: 57
Hidden sizes: [50, 25]
Output size: 10
Epoch [100/1000], Loss: 0.0940, Accuracy: 0.1150
Epoch [200/1000], Loss: 0.0924, Accuracy: 0.1220
Epoch [300/1000], Loss: 0.0917, Accuracy: 0.1220
Epoch [400/1000], Loss: 0.0911, Accuracy: 0.1300
Epoch [500/1000], Loss: 0.0906, Accuracy: 0.1470
Epoch [600/1000], Loss: 0.0902, Accuracy: 0.1500
Epoch [700/1000], Loss: 0.0904, Accuracy: 0.1210
Epoch [800/1000], Loss: 0.0903, Accuracy: 0.1270
Epoch [900/1000], Loss: 0.0903, Accuracy: 0.1290
Epoch [1000/1000], Loss: 0.0902, Accuracy: 0.1290

First 5 elements of the dataset (features and labels):
0    blues
1    blues
2    blues
3    blues
4    blues
Name: label, dtype: object

First 5 predicted values:
['country' 'country' 'country' 'country' 'country']
